In [28]:
# from pprint import pprint                                               # Alt + Enter   :: runs currently selected cell and insert new cell immediately below

import requests                                                         # Jupyter keyboard shortcuts cheatsheet 
import bs4 as bs                                                        # Ctrl + Enter  :: runs currently selected cell, focus remains in current cell
import lxml                                                             # Shift + Enter :: runs current cell and insert new one immediately below
import html5lib                                                         # Esc Shift + L :: Toggle line numbering for full notebook

httpaddr = "http://www.ambito.com"                                      # Esc dd        :: (Command mode) Esc *FOLLOWED* by D D. Erases current cell (pretty much like a line on Vim)
requests_ret = requests.get(httpaddr)                                   # Alt + <arrow> :: Move code cell
                                                                        # Esc M         :: (Command mode) Switch to typing markdown (on a new cell)
soup_html5lib = bs.BeautifulSoup(requests_ret.text, 'html5lib')         # Esc Y         :: (Command mode) Switch to typing code (on a new cell)
soup_lxml = bs.BeautifulSoup(requests_ret.text, 'lxml')                 # Esc L         :: Toggle line numbering for present cell
                                                                        
soup = soup_lxml    # parse both - choose one
# soup = soup_html5lib    # parse both - choose one

